In [27]:
import requests
from datetime import datetime
import time
import json
import glob

import pandas as pd
import numpy as np
import geopandas as gpd
from pytz import timezone
%matplotlib inline
import seaborn as sns
from tqdm import tqdm_notebook as tqdm

In [3]:
api_key = 'ac31e02046794bd3f959fbc98fc25231'

In [101]:
gdf = gpd.read_file('countries/SRB/shapefiles/SRB_03M_LVL3.geojson')
center = gpd.GeoDataFrame(gdf[['NUTS_ID', 'NAME_LATN']])
center['geometry'] = gdf.centroid
center = center.to_crs(gdf.crs)
center['lat'] = center.geometry.y
center['lon'] = center.geometry.x
center
# gdf

C:\Users\sonle\AppData\Local\Temp\ipykernel_15480\1180128645.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center['geometry'] = gdf.centroid
C:\Users\sonle\AppData\Local\Temp\ipykernel_15480\1180128645.py:3: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  center['geometry'] = gdf.centroid


,NUTS_ID,NAME_LATN,geometry,lat,lon
0,RS222,Braničevska oblast,POINT (21.52958 44.46493),44.464932,21.529579
1,RS223,Zaječarska oblast,POINT (22.14882 43.74560),43.745597,22.148822
2,RS224,Jablanička oblast,POINT (21.89568 42.91891),42.918908,21.895678
3,RS221,Borska oblast,POINT (22.25064 44.31574),44.315736,22.250639
4,RS214,Moravička oblast,POINT (20.30262 43.77579),43.775787,20.302622
5,RS215,Pomoravska oblast,POINT (21.36354 43.98853),43.988525,21.363544
6,RS216,Rasinska oblast,POINT (21.16570 43.51534),43.515339,21.165701
7,RS217,Raška oblast,POINT (20.55523 43.37183),43.371835,20.555231
8,RS218,Šumadijska oblast,POINT (20.77925 44.11715),44.117152,20.779246
9,RS110,City of Belgrade,POINT (20.41785 44.64808),44.648080,20.417850


In [104]:
gid_1 = center['NUTS_ID'].to_list()
lat = center['lat'].to_list()
lon = center['lon'].to_list()

loc_dict = {}
for i in range(len(gid_1)):
    loc_dict[gid_1[i]] = [lat[i], lon[i]]

In [105]:
loc_dict

{'RS222': [44.464932303207775, 21.529579403451127],
 'RS223': [43.74559679862961, 22.148821926387672],
 'RS224': [42.91890842540353, 21.895678409152335],
 'RS221': [44.315736374959805, 22.250638920103725],
 'RS214': [43.77578650262159, 20.302621857776536],
 'RS215': [43.98852539405567, 21.3635437228867],
 'RS216': [43.51533874558893, 21.165700797216388],
 'RS217': [43.37183495924805, 20.555231225462038],
 'RS218': [44.11715246791616, 20.779246272154392],
 'RS110': [44.648079734707906, 20.417850485007214],
 'RS225': [43.4159833110961, 21.867412040012155],
 'RS226': [43.14449640246478, 22.558835013752265],
 'RS227': [44.45746887746036, 20.971323440266044],
 'RS228': [42.533081211839715, 22.07296379479984],
 'RS229': [43.164360599753074, 21.381014652659935],
 'RS121': [45.71252352074958, 19.21191524566298],
 'RS122': [45.00592425845587, 20.978338147729204],
 'RS123': [45.40019584357676, 19.736510020838253],
 'RS124': [45.90684549037046, 20.192564309219172],
 'RS125': [45.92525444974453, 1

In [109]:
def create_json(loc:str, start: int, end: int) -> dict:
    lat = loc_dict[loc][0]
    lon = loc_dict[loc][1]
    hist_url = f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}'
    r = requests.get(hist_url)
    json_out = r.json()
    json_out['name'] = loc

    return json_out

year = 2023
start_date = int(time.mktime(datetime(year, 1, 1).timetuple()))
end_date = int(time.mktime(datetime(year, 12, 31).timetuple()))

for location_name in list(loc_dict.keys()):
    print(location_name)
    json_res = create_json(location_name, start_date, end_date)
    with open(f"countries/SRB/data/air_pollution/district/{location_name}_{year}.json", "w") as outfile: 
            json.dump(json_res, outfile)




RS222
RS223
RS224
RS221
RS214
RS215
RS216
RS217
RS218
RS110
RS225
RS226
RS227
RS228
RS229
RS121
RS122
RS123
RS124
RS125
RS126
RS127
RS211
RS212
RS213


In [111]:
# gdf = gpd.read_file('countries/SRB/gadm41_SRB_2.json')
# center = gpd.GeoDataFrame(gdf[['GID_2', 'NAME_2']])
# center['geometry'] = gdf.centroid
# center = center.to_crs(gdf.crs)
# center['lat'] = center.geometry.y
# center['lon'] = center.geometry.x

# gid_2 = center['GID_2'].to_list()
# lat = center['lat'].to_list()
# lon = center['lon'].to_list()

# loc_dict = {}
# for i in range(len(gid_2)):
#     loc_dict[gid_2[i]] = [lat[i], lon[i]]

# loc_dict

In [112]:
# def create_json(loc:str, start: int, end: int) -> dict:
#     lat = loc_dict[loc][0]
#     lon = loc_dict[loc][1]
#     hist_url = f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}'
#     r = requests.get(hist_url)
#     json_out = r.json()
#     json_out['name'] = loc

#     return json_out

# year = 2023
# start_date = int(time.mktime(datetime(year, 1, 1).timetuple()))
# end_date = int(time.mktime(datetime(year, 12, 31).timetuple()))

# for location_name in list(loc_dict.keys()):
#     print(location_name)
#     json_res = create_json(location_name, start_date, end_date)
#     with open(f"countries/SRB/air_pollution/municipalities/{location_name}_{year}.json", "w") as outfile: 
#             json.dump(json_res, outfile)




In [114]:
paths = glob.glob('D:\Work\WB\LDT\countries\SRB\data\\air_pollution\district/*.json')
paths.sort()
paths

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\sonle\AppData\Local\Temp\ipykernel_15480\689126405.py:1: SyntaxWarning: invalid escape sequence '\W'
  paths = glob.glob('D:\Work\WB\LDT\countries\SRB\data\\air_pollution\district/*.json')


['D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_pollution\\district\\RS110_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_pollution\\district\\RS121_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_pollution\\district\\RS122_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_pollution\\district\\RS123_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_pollution\\district\\RS124_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_pollution\\district\\RS125_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_pollution\\district\\RS126_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_pollution\\district\\RS127_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_pollution\\district\\RS211_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_pollution\\district\\RS212_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_pollution\\district\\RS213_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\data\\air_polluti

In [115]:
dfs = []

for path in tqdm(paths):
    # print(path)
    with open(path) as f:
        d = json.load(f)
    i = path.rindex('\\')

    no2 = [x['components']['no2'] for x in d['list']]
    so2 = [x['components']['so2'] for x in d['list']]
    co = [x['components']['co'] for x in d['list']]
    o3 = [x['components']['o3'] for x in d['list']]
    pm10 = [x['components']['pm10'] for x in d['list']]
    pm25 = [x['components']['pm2_5'] for x in d['list']]
    dt = [x['dt'] for x in d['list']]

    df = pd.DataFrame({'datetime_unix': dt,
                    'o3': o3,
                    'pm25': pm25,
                    'pm10': pm10,
                    'co': co,
                    'so2': so2,
                    'no2': no2})
    df['lon'] = d['coord']['lon']
    df['lat'] = d['coord']['lat']
    df['location'] = path[i+1:-10]
    dfs.append(df)

full_df = pd.concat(dfs, axis=0).reset_index(drop=True)
full_df['timestamp'] = pd.to_datetime(full_df['datetime_unix'],unit='s')
full_df['timestamp'] = full_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('Asia/Ho_Chi_Minh')
full_df = full_df.drop(['datetime_unix'], axis=1)
full_df = full_df[['location', 'lon', 'lat', 'timestamp',
                   'o3', 'pm25', 'pm10', 'co', 'so2', 'no2']]
full_df = full_df.sort_values(by=['location', 'timestamp']).reset_index(drop=True)
full_df

C:\Users\sonle\AppData\Local\Temp\ipykernel_15480\1850091446.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for path in tqdm(paths):


  0%|          | 0/25 [00:00<?, ?it/s]

,location,lon,lat,timestamp,o3,pm25,pm10,co,so2,no2
0,RS110,20.4179,44.6481,2023-01-01 00:00:00+07:00,5.32,18.34,28.06,257.02,0.15,16.62
1,RS110,20.4179,44.6481,2023-01-01 01:00:00+07:00,8.49,19.64,28.64,257.02,0.18,15.59
2,RS110,20.4179,44.6481,2023-01-01 02:00:00+07:00,39.34,17.57,22.20,230.31,0.73,6.94
3,RS110,20.4179,44.6481,2023-01-01 03:00:00+07:00,40.05,17.59,22.24,230.31,0.55,6.00
4,RS110,20.4179,44.6481,2023-01-01 04:00:00+07:00,38.98,17.74,22.59,226.97,0.42,5.48
...,...,...,...,...,...,...,...,...,...,...
215420,RS229,21.3810,43.1644,2023-12-30 20:00:00+07:00,44.70,19.62,22.71,300.41,0.32,3.60
215421,RS229,21.3810,43.1644,2023-12-30 21:00:00+07:00,42.20,20.66,24.22,300.41,0.29,4.24
215422,RS229,21.3810,43.1644,2023-12-30 22:00:00+07:00,41.48,20.89,24.84,300.41,0.28,5.06
215423,RS229,21.3810,43.1644,2023-12-30 23:00:00+07:00,41.13,20.79,25.07,300.41,0.27,5.91


In [116]:
full_df['year'] = full_df['timestamp'].dt.year
full_df

,location,lon,lat,timestamp,o3,pm25,pm10,co,so2,no2,year
0,RS110,20.4179,44.6481,2023-01-01 00:00:00+07:00,5.32,18.34,28.06,257.02,0.15,16.62,2023
1,RS110,20.4179,44.6481,2023-01-01 01:00:00+07:00,8.49,19.64,28.64,257.02,0.18,15.59,2023
2,RS110,20.4179,44.6481,2023-01-01 02:00:00+07:00,39.34,17.57,22.20,230.31,0.73,6.94,2023
3,RS110,20.4179,44.6481,2023-01-01 03:00:00+07:00,40.05,17.59,22.24,230.31,0.55,6.00,2023
4,RS110,20.4179,44.6481,2023-01-01 04:00:00+07:00,38.98,17.74,22.59,226.97,0.42,5.48,2023
...,...,...,...,...,...,...,...,...,...,...,...
215420,RS229,21.3810,43.1644,2023-12-30 20:00:00+07:00,44.70,19.62,22.71,300.41,0.32,3.60,2023
215421,RS229,21.3810,43.1644,2023-12-30 21:00:00+07:00,42.20,20.66,24.22,300.41,0.29,4.24,2023
215422,RS229,21.3810,43.1644,2023-12-30 22:00:00+07:00,41.48,20.89,24.84,300.41,0.28,5.06,2023
215423,RS229,21.3810,43.1644,2023-12-30 23:00:00+07:00,41.13,20.79,25.07,300.41,0.27,5.91,2023


In [117]:
gdf

,id,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry
0,RS222,RS222,3,RS,Braničevska oblast,Браничевска област,4,3,3,RS222,"POLYGON ((21.82202 44.08251, 21.80861 44.09600..."
1,RS223,RS223,3,RS,Zaječarska oblast,Зајечарска област,4,2,3,RS223,"POLYGON ((22.33758 43.38527, 22.34016 43.41702..."
2,RS224,RS224,3,RS,Jablanička oblast,Јабланичка област,4,2,3,RS224,"POLYGON ((22.22691 43.08012, 22.25253 43.06249..."
3,RS221,RS221,3,RS,Borska oblast,Борска област,4,2,3,RS221,"POLYGON ((22.53615 44.04551, 22.49600 44.04771..."
4,RS214,RS214,3,RS,Moravička oblast,Моравичка област,4,2,3,RS214,"POLYGON ((20.29580 43.31169, 20.28285 43.32850..."
5,RS215,RS215,3,RS,Pomoravska oblast,Поморавска област,4,3,3,RS215,"POLYGON ((21.14757 44.31517, 21.21143 44.30156..."
6,RS216,RS216,3,RS,Rasinska oblast,Расинска област,4,3,3,RS216,"POLYGON ((20.92356 43.14608, 20.83927 43.17651..."
7,RS217,RS217,3,RS,Raška oblast,Рашка област,4,2,3,RS217,"POLYGON ((20.92512 43.83154, 20.93004 43.81211..."
8,RS218,RS218,3,RS,Šumadijska oblast,Шумадијска област,4,2,3,RS218,"POLYGON ((20.78354 44.32662, 20.83002 44.31521..."
9,RS110,RS110,3,RS,City of Belgrade,Београдска област,4,1,3,RS110,"POLYGON ((20.78354 44.32662, 20.71708 44.35672..."


In [118]:
gb = full_df.groupby(['location', 'year']).agg({'pm25': 'mean',
                                           'pm10': 'mean',
                                           'no2': 'mean'}).reset_index()
gb = gb.rename({'location': 'NUTS_ID'}, axis=1)
gb = gdf[['NUTS_ID', 'NAME_LATN']].merge(gb, on=['NUTS_ID'], how='left')
gb = gb.rename({'NAME_LATN': 'DISTRICT'}, axis=1)
gb.columns = gb.columns.str.upper()
gb['UNIT'] = 'ug/m3'
gb

,NUTS_ID,DISTRICT,YEAR,PM25,PM10,NO2,UNIT
0,RS222,Braničevska oblast,2023,8.545324,10.240347,2.479756,ug/m3
1,RS223,Zaječarska oblast,2023,8.389348,10.489712,3.082540,ug/m3
2,RS224,Jablanička oblast,2023,9.449128,12.618604,3.626326,ug/m3
3,RS221,Borska oblast,2023,8.248888,9.779135,2.887685,ug/m3
4,RS214,Moravička oblast,2023,8.329545,10.718192,3.116211,ug/m3
5,RS215,Pomoravska oblast,2023,9.353810,11.594723,4.111527,ug/m3
6,RS216,Rasinska oblast,2023,9.954719,12.808694,3.584845,ug/m3
7,RS217,Raška oblast,2023,8.121223,10.758277,3.984768,ug/m3
8,RS218,Šumadijska oblast,2023,10.146838,12.556503,4.107872,ug/m3
9,RS110,City of Belgrade,2023,11.444601,15.345293,7.303607,ug/m3


In [119]:
gb.to_csv('countries/SRB/datasets/SRB_air_pollution_2023_NUTS3.csv', index=False)

In [89]:
gdp = pd.read_csv('countries/SRB/data/gdp/EU_GDP_2021_PPP.csv')
gdp

,VERSIONS,LEVL_CODE,NUTS_ID,NUTS_NAME,YEAR,DATE,UNIT,VALUE
0,2021,0,AL,SHQIPËRIA,2008,2008,Million EUR,8800.29
1,2021,0,AL,SHQIPËRIA,2009,2009,Million EUR,8662.26
2,2021,0,AL,SHQIPËRIA,2010,2010,Million EUR,8996.62
3,2021,0,AL,SHQIPËRIA,2011,2011,Million EUR,9268.33
4,2021,0,AL,SHQIPËRIA,2012,2012,Million EUR,9585.82
...,...,...,...,...,...,...,...,...
153260,2021,3,SK042,Kosický kraj,2023,2023,Million PPS,17376.46
153261,2021,3,SK042,Kosický kraj,2024,2024,Million PPS,18252.13
153262,2021,3,SKZZZ,Extra regio (SK),2022,2022,Million PPS,0.00
153263,2021,3,SKZZZ,Extra regio (SK),2023,2023,Million PPS,0.00


In [124]:
srb_gdp = gdp[(gdp['NUTS_ID'].str.contains('RS')) & (gdp['LEVL_CODE'] == 3)]
srb_gdp = srb_gdp[srb_gdp['NUTS_ID'] != 'RSZZZ']
srb_gdp = srb_gdp[srb_gdp['UNIT'] == 'Million EUR']
srb_gdp = srb_gdp[srb_gdp['YEAR'] == 2023].reset_index(drop=True)
srb_gdp['VALUE'] = srb_gdp['VALUE']*1000000
srb_gdp['UNIT'] = 'EUR 2021'
srb_gdp = pd.merge(srb_gdp, gdf[['NUTS_ID', 'NAME_LATN']], on=['NUTS_ID'], how='left')
srb_gdp = srb_gdp.rename({'VALUE': 'GDP',
                          'NAME_LATN': 'DISTRICT'}, axis=1)
srb_gdp = srb_gdp[['NUTS_ID', 'DISTRICT', 'YEAR', 'GDP', 'UNIT']]
srb_gdp.head()

,NUTS_ID,DISTRICT,YEAR,GDP,UNIT
0,RS110,City of Belgrade,2023,2.788723e+10,EUR 2021
1,RS121,Zapadnobačka oblast,2023,1.272390e+09,EUR 2021
2,RS122,Južnobanatska oblast,2023,2.453800e+09,EUR 2021
3,RS123,Južnobačka oblast,2023,7.281000e+09,EUR 2021
4,RS124,Severnobanatska oblast,2023,1.055790e+09,EUR 2021


In [126]:
srb_gdp.to_csv('countries/SRB/datasets/SRB_gdp_2023_NUTS3.csv', index=False)